<a href="https://colab.research.google.com/github/sinansonmez/Coursera_Capstone/blob/master/immobiliare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [26]:
# Use the requests library to download the webpage
base_url = "https://www.immobiliare.it/vendita-case/"
municipality = "lainate"
page = "/?pag=1"
url = base_url + municipality + page

html_data = requests.get(url).text

In [27]:
# Parse the html data using beautiful_soup
soup = BeautifulSoup(html_data, "html5lib")
page_num = soup.find_all('li', class_='disabled')[-1]

In [28]:
# Using beautiful soup extract the table and store it into a dataframe named istanbul_data. 
# The dataframe have columns. Fill in each variable with the correct data from the list cell.

# TODO id ekle en basa

real_estate_data = pd.DataFrame(columns=['title', 'price', 'room', 'sqm', 'bagni', 'piano', 'link'])

list = soup.find_all('div', class_='listing-item_body--content')

for item in list:
  title = item.find('p').text.replace('\n','')
  link = item.find('a').get('href').replace('\n','')
  for detail in item.find_all('li'):
    if (detail.text.find('€') != -1):
      price = detail.text.replace('€ ','').replace('.','').replace(' ','').replace('da','').replace('\n','')
    if (detail.text.find('locali') != -1):
      room = detail.text.replace(' ', '').replace('locali', '').replace('\n','')
    if (detail.text.find('superfici') != -1):
      sqm = detail.text.replace('m2superficie', '').replace('\n','')
    if (detail.text.find('bagni') != -1):
      bagni = detail.text.replace('bagni', '').replace('\n','')
    if (detail.text.find('piano') != -1):
      piano = detail.text.replace('piano', '').replace('\n','')
  
  real_estate_data = real_estate_data.append({'title': title, 'price': price, 'room': room, 'sqm': sqm, 'bagni': bagni, 'piano': piano, 'link': link}, ignore_index=True)

real_estate_data.head()
# real_estate_data.to_csv(index=False)
  




,title,price,room,sqm,bagni,piano,link
0,Tr...,130000,3,105,1,1,https://www.immobiliare.it/annunci/86865074/
1,Tr...,167000,3,90,2,1,https://www.immobiliare.it/annunci/86837136/
2,At...,525000,5,220,3,3,https://www.immobiliare.it/annunci/86849162/
3,Vi...,469000,5,250,3,3,https://www.immobiliare.it/annunci/86820032/
4,Tr...,137000,3,100,2,2,https://www.immobiliare.it/annunci/86808698/
